# 0. General Imports

In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Deloitte Backup/Financial Data Clustering Paper/"

In [5]:
es600 = pd.read_csv(f"{path}230507_es600_final.csv")

# 1. Data Pre-Processing

In [6]:
es600.shape[0]

7801

In [7]:
#filtering for derivedEmissions in the STOXX600 dataset
es600 = es600.loc[es600.derivedEmissions.notna()]

In [8]:
es600.shape[0]

6239

In [9]:
#applying condition that 80% at least for one column must be non-null and drop the rest of columns
cond = es600.count() < es600.shape[0] * 0.8
drop_columns = es600.count()[cond].index.to_list()
es600 = es600.drop(columns = drop_columns)
drop_columns

['Environmental Expenditures',
 'Environmental Provisions',
 'CO2e Indirect Emissions, Scope 3',
 'Environmental R&D Expenditures',
 'Direct Energy/Energy Purchased Direct',
 'Direct Energy/Energy Produced Direct',
 'Indirect Energy Use',
 'researchAndDevelopment',
 'shortTermInvestments',
 'rnd/sales',
 'capex/fixedAssets',
 'sga/sales']

In [12]:
def encode_categorical_variables(data: pd.DataFrame, variables: list):

  end_encoded_df = pd.DataFrame()

  for variable in variables:

    encoded_df = pd.get_dummies(data[[variable]].copy())
    end_encoded_df = pd.concat([end_encoded_df,encoded_df], axis = 1)

  data = data.drop(columns = variables)
  data = pd.concat([data, end_encoded_df], axis=1)

  return data

In [13]:
data = encode_categorical_variables(data = es600.copy(), variables = ["econIdDesc","IncomeGroup"])

# 3. Conducting Initial Test of Clustering with UMAP

In [10]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.9 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=c5b88700823b4ea5de23d0c103afb65876e389ba1ca36cc08a4f9cb082aff5c9
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [11]:
import umap
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
len(data)

6239

In [33]:
data.isna().sum().sort_values(ascending = False).head(25)

Energy Use Total                                             1096
cogs/sales                                                    945
grossMargin                                                   943
grossProfit                                                   911
capex/totalSales                                              829
CO2 Equivalents Emission Indirect                             605
CO2 Equivalents Emission Direct                               570
capex/totalAssets                                             448
propertyPlantEquipmentTotal                                   307
totalAssets/commonEquity                                      141
ebit/interest                                                 113
Emission Reduction Objectives/Targets Emissions Reduction      91
totalAssets/employee                                           91
roa                                                            78
sales/employee                                                 40
employees 

In [34]:
data = data.dropna()

In [35]:
data.head(1)

,OrgId,econId,OrgName_x,FisYear,Id_A4,OrgName_y,BusTRBCId,Emission Reduction Processes/Policy Emissions Reduction,Emission Reduction Objectives/Targets Emissions Reduction,derivedEmissions,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,emissionIntensity,Energy Use Total,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score,totalRevenue,totalAssets,employees,grossProfit,operatingIncome,earningsBeforeTax,propertyPlantEquipmentTotal,totalDebt,totalLiabilities,commonStock,netIncome,ebitda,enterpriseValue,totalAssets/commonEquity,ebit/interest,grossMargin,cf/sales,operatingMargin,roa,cogs/sales,sales/employee,totalAssets/employee,capex/totalAssets,capex/totalSales,capex,ISOCountryCodeDerived,Region,GDP PPP,econIdDesc_Basic Materials,econIdDesc_Consumer Cyclicals,econIdDesc_Consumer Non-Cyclicals,econIdDesc_Energy,econIdDesc_Financials,econIdDesc_Healthcare,econIdDesc_Industrials,econIdDesc_Real Estate,econIdDesc_Technology,econIdDesc_Utilities,IncomeGroup_High income,IncomeGroup_Upper middle income
1,10836.0,54.0,Siemens AG,2009,@SIEME18,Siemens AG,5440.0,1.0,1.0,2574000.0,2574000.0,1115000.0,1459000.0,22.943176,23800000.0,0.88308,0.774873,0.666667,0.925,0.977273,0.775,0.982143,0.807692,0.875,0.613636,0.993506,0.967532,0.951613,112190.219438,138938.419204,405000.0,30452.697364,8699.934304,5695.061302,40153.768119,28743.154418,94182.936171,4014.791352,3354.6853,12719.116605,63156.422093,3.43898,6.69693,27.14381,9.35539,7.75463,2.97482,69.27372,189261.7284,226259.25926,3.19614,3.81339,4278.248313,DEU,Europe & Central Asia,36842.916198,0,0,1,0,0,0,0,0,0,0,1,0


In [36]:
data.to_csv("capstone_dataset.csv", index = False)

## v1 - esg features

In [15]:
esg_features = ['derivedEmissions', 'emissionIntensity', 'Energy Use Total', 'ESG SCORE',
                'ESG Combined Score', 'ESG Controversies Score', 'Resource Use Score',
                'Emissions Score', 'Environmental Innovation Score', 'Workforce Score',
                'Human Rights Score', 'Community Score', 'Product Responsibility Score',
                'Management Score', 'Shareholders Score', 'CSR Strategy Score']

In [16]:
esg_data = data[esg_features]
esg_data = esg_data.replace([np.inf, -np.inf], np.nan)
esg_data = esg_data.dropna()
if esg_data.isna().any().any():
    print("There are NaN values in the DataFrame.")
else:
    print("No NaN values found.")

No NaN values found.
